In [1]:
from __future__ import division, print_function
import os
import numpy as np
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import pickle
import operator

In [2]:
app = Flask(__name__)

dataset = []
def loadDataset(filename):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break

loadDataset("my.dat")

In [3]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    
    #Method to calculate distance between two instances.
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    print("distane is",distance)
    return distance


In [4]:
def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    print("neighbors is ",neighbors)
    return neighbors   

In [5]:
def nearestClass(neighbors):
    classVote ={}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1 
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    print("sorter is ",sorter)
    return sorter[0][0]

In [6]:
print('Model loaded. Check http://127.0.0.1:5000/')

Model loaded. Check http://127.0.0.1:5000/


In [7]:

@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('music.html')

In [8]:
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['image']
        
        # Save the file to ./uploads
        basepath = "E:/Externship/Flask"
        file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        print(file_path)
        i=1
        
        results = {1: 'blues', 2: 'classical', 3: 'country', 4: 'disco', 5: 'hiphop', 
                   6: 'jazz', 7: 'metal', 8: 'pop', 9: 'reggae', 10: 'rock'}
    
        (rate,sig)=wav.read(file_path)
        print(rate,sig)
        mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature=(mean_matrix,covariance,0)
        pred=nearestClass(getNeighbors(dataset ,feature , 8))
        
        print("predicted genre = ",pred,"class = ",results[pred])
        return "This song is classified as a "+str(results[pred])

In [ ]:

if __name__ == '__main__':
    app.run(threaded = False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Jul/2023 18:26:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2023 18:26:48] "GET /favicon.ico HTTP/1.1" 404 -


E:/Externship/Flask\uploads\classical.wav
22050 [-1326 -2561 -2863 ... -2148 -1412  -965]
distane is 28.524543707256612
distane is 20.67965127964911
distane is 39.42670415695028
distane is 21.13219931234105
distane is 33.76092808941871
distane is 25.103416458816994
distane is 31.979200579115755
distane is 21.6485943775727
distane is 28.663116852366763
distane is 16.388240101020294
distane is 40.660136278413425
distane is 24.684433010481172
distane is 58.322944040308926
distane is 22.7094705657274
distane is 47.41343410224249
distane is 23.6128018205494
distane is 38.11966247708887
distane is 41.427206033911986
distane is 37.34015062180848
distane is 20.68339978630072
distane is 37.96638021611366
distane is 17.570555395140975
distane is 43.85320080931848
distane is 24.4191444583619
distane is 38.23433261460302
distane is 24.820826522303527
distane is 37.90742454493492
distane is 63.29094580579971
distane is 52.96232241742079
distane is 25.00961174366526
distane is 35.76704436512869
dist

127.0.0.1 - - [17/Jul/2023 18:27:04] "POST /predict HTTP/1.1" 200 -


distane is 63.743233450101656
distane is 85.24170365765019
distane is 86.69906647001486
distane is 53.601833692209844
distane is 22.451882491231206
distane is 63.35906464065563
distane is 57.67247623419236
distane is 101.19429704600337
distane is 195.67595179370954
distane is 106.1767691241315
distane is 63.56925059543677
distane is 103.64364765740359
distane is 67.76208224628994
distane is 116.02502004452806
distane is 87.89216703154229
distane is 85.87149373053612
distane is 53.801110247359844
distane is 66.00285474852159
distane is 54.97801116455306
distane is 65.46779721246628
distane is 28.16035486201845
distane is 82.86836993144732
distane is 76.54430015391634
distane is 56.806402865613734
distane is 29.09596739242228
distane is 94.36962411262598
distane is 32.20632758226339
distane is 68.13502187187314
distane is 42.61896530688432
distane is 59.70596385821602
distane is 42.695424795677646
distane is 38.47033029574603
distane is 17.572598961308735
distane is 35.975778536326885
di

...

127.0.0.1 - - [17/Jul/2023 18:32:30] "POST /predict HTTP/1.1" 200 -


distane is 94.73446717079767
distane is 32.53697390883759
distane is 59.403747042323516
distane is 35.333466986687185
distane is 106.72144042101995
distane is 23.884053568555387
distane is 94.15643610797696
distane is 26.935417764587577
distane is 110.50248243723246
distane is 26.535408272926503
distane is 56.979611422129224
distane is 31.80829909547071
distane is 113.71334671358582
distane is 30.48442160228594
distane is 57.26924243340869
distane is 28.597838718897357
distane is 237.5938684687843
distane is 42.275490101066254
distane is 33.62465023266159
distane is 37.15697935398532
distane is 91.94110983814376
distane is 31.711290307738103
distane is 35.75053921992637
distane is 21.544536628118706
distane is 34.10124375698019
distane is 16.742557478270843
distane is 68.62675125518632
distane is 35.17449635864163
distane is 267.03241268249315
distane is 40.34279556215816
distane is 52.93394646956565
distane is 20.215887135602596
distane is 59.52667748327133
distane is 24.9788611959425

E:/Externship/Flask\uploads\rock.00010.wav
22050 [-2461 -2989 -1841 ...  5478  4676  3698]
distane is 15.42901819289574
distane is 13.688497171509493
distane is 23.60302147318461
distane is 16.270135281665425
distane is 17.346889876793856
distane is 13.076141738003866
distane is 21.334269039484482
distane is 17.145989260771252
distane is 28.07117677651376
distane is 30.673669581879672
distane is 25.756085597599075
distane is 23.560183823021
distane is 37.8914829549607
distane is 27.0619497714277
distane is 30.74232137167816
distane is 20.513893875614663
distane is 14.86635035027264
distane is 13.114925243614156
distane is 35.18112531671977
distane is 37.967181149765324
distane is 34.244611449007614
distane is 25.304757799007284
distane is 33.629729838666094
distane is 21.519452356223425
distane is 35.423917622695626
distane is 43.48757634847068
distane is 32.63896761476017
distane is 79.1770863586149
distane is 53.59528240922107
distane is 82.0940961612849
distane is 38.97976519208319


127.0.0.1 - - [17/Jul/2023 18:32:42] "POST /predict HTTP/1.1" 200 -


distane is 46.750353215429634
distane is 42.42104185950284
distane is 52.954194368206956
distane is 51.02738450374207
distane is 43.18923446031503
distane is 25.968746555964707
distane is 43.595630855316195
distane is 50.87397278327115
distane is 44.734344816930054
distane is 34.32969579524417
distane is 62.28598569292514
distane is 62.175288578223544
distane is 55.73102810862906
distane is 56.77964855997206
distane is 56.315500978435594
distane is 35.94927656347314
distane is 56.315500978435594
distane is 35.94927656347314
distane is 70.50636334758397
distane is 48.76117196705743
distane is 40.64368809731698
distane is 33.99897031745768
distane is 56.63446770449437
distane is 67.20285209231375
distane is 89.9880375409578
distane is 192.18543727662606
distane is 67.49059021732003
distane is 40.08788050250244
distane is 63.11486587190106
distane is 77.56915091673812
distane is 42.42232154887824
distane is 52.0288788514927
distane is 48.799600779462104
distane is 41.0129835115111
distane